In [97]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [98]:
messages = pd.read_csv('SMSSpamCollection.csv', sep='\t', names=['labels', 'message'])

In [99]:
messages

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [100]:
stop_words = set(stopwords.words('english'))
snowball_stemmer = SnowballStemmer('english')

In [101]:
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [snowball_stemmer.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

In [102]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkts st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breath

In [103]:
y = pd.get_dummies(messages['labels'], dtype=int)
y = y.iloc[:, 1].values

In [104]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42)

In [105]:
X_test

['squeeeeez christma hug u lik frndshp den hug back u get u r cute u r luvd u r lucki none peopl hate u',
 'also sorta blown coupl time recent id rather text blue look weed',
 'mmm that better got roast b better drink good indian',
 'mm kanji dont eat anyth heavi ok',
 'ring come guy costum gift futur yowif hint hint',
 'sari need tim bollox hurt lot tol',
 'love decis feel could decid love life would much simpler less magic',
 'supervisor find one lor thk student havent ask yet tell u aft ask',
 'dear good morn',
 'chennai velacheri',
 'lol grr mom take forev prescript pharmaci like minut away ugh',
 'valentin huh proof fb page ugh glad realli watch rupaul show tool',
 'wif buy tix lar',
 'er hello thing quit go plan limp slowli home follow aa exhaust hang',
 'free rington text first poli text get true tone help st free tone x pw e nd txt stop',
 'sir wait call',
 'crazi ar marri like gd look guy fren like say korean leona fave dun thk aft think mayb prob go',
 'hey free call',
 'figh

In [106]:
tf_idf = TfidfVectorizer(max_features=2500, ngram_range=(1, 2))
X_train = tf_idf.fit_transform(X_train).toarray()
X_test = tf_idf.transform(X_test).toarray()

In [107]:
spam_tfidf_model = MultinomialNB().fit(X_train, y_train)

In [108]:
y_preds = spam_tfidf_model.predict(X_test)

In [109]:
report = classification_report(y_test, y_preds)
acc_score = accuracy_score(y_test, y_preds)
print(report)
print(acc_score)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.98      0.85      0.91       149

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.97847533632287
